In [27]:
from dotenv import load_dotenv, find_dotenv
from model_alignment.model_helper import ModelHelper
from model_alignment.alignable_model_calls import ConversationTurn
from openai import OpenAI
from typing import List, Optional, Union
from model_alignment.single_run import AlignableSingleRun
# Load environment variables
print(load_dotenv(find_dotenv()))


import json

# Load environment variables
print(load_dotenv(find_dotenv()))

class ChatGPTModelHelper(ModelHelper):
    def __init__(self):
        """Initialize the OpenAI client."""
        self.client = OpenAI()
        
    def predict(
        self, 
        prompt: str, 
        temperature: float = 0, 
        stop_sequences: Optional[List[str]] = None, 
        candidate_count: int = 1,  
        max_output_tokens: Optional[int] = None
    ) -> str:
        """
        Implements the predict method for single-run calls.
        Returns a single string response.
        """
        try:
            print("\n=== Debug: Incoming Prompt ===")
            print(f"Prompt: {prompt}") 
            response = self.client.chat.completions.create(
                model="gpt-4-turbo-preview",
                messages=[{"role": "user", "content": prompt}],
                temperature=temperature,
                n=1,
                max_tokens=4000,  # Increased token limit
                stop=stop_sequences
            )
            content = response.choices[0].message.content
            print("\n=== Debug: Model Response ===")
            print(f"Response: {content}")  # Print first 200 chars of response
            print("\n=== Debug: Model Response End ===")
            
            return content
            
        except Exception as e:
            raise Exception(f"OpenAI API call failed: {str(e)}")

def display_response(response):
    """Helper function to properly display response content."""
    if isinstance(response, ConversationTurn):
        # Access all attributes of ConversationTurn
        attributes = vars(response)
        if 'text' in attributes:
            print(attributes['text'])
        elif 'response' in attributes:
            print(attributes['response'])
        else:
            print("Response content not found in expected attributes.")
            print("Available attributes:", attributes)
    else:
        print(response)

def run_gen_ai_course_creation():
    model_helper = ChatGPTModelHelper()

    initial_prompt = """Create a course for GEN AI from beginner to advance, along with the resources 
    and time durations. Look into MOOCs, Book, YouTube channels, Papers, GitHub"""

    alignable_prompt = AlignableSingleRun(
        prompt_model=model_helper
    )
    alignable_prompt.set_model_description(initial_prompt)

    try:
        # Generate and display initial response
        response = alignable_prompt.send_input()


        # User feedback
        user_feedback = """What are the critical components of determining GEN AI course. 
        Base your prompt on critical components"""

        # Generate and display principles
        principles = alignable_prompt.rewrite_response(user_feedback)

        # Update model description
        if isinstance(principles, ConversationTurn):
            principles_text = principles.text if hasattr(principles, 'text') else str(principles)
        else:
            principles_text = str(principles)
                # Show the principle and prompts before improved response
        print("\nGenerated Principles:")
        print("="*50)
        print(alignable_prompt.data.principles)  # Shows raw principles
        
        print("\nBase Prompt After Principles:")
        print("="*50)
        print(alignable_prompt.get_model_description())  # Shows how principles modified base prompt
        
        print("\nFull Prompt with Principles:")
        print("="*50)
        print(alignable_prompt.get_model_description_with_principles())  # Shows prompt + principles

        alignable_prompt.set_model_description(alignable_prompt.get_model_description_with_principles())
        print("\nImproved Response:")
        improved_response = alignable_prompt.send_input()
        display_response(improved_response)
        

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        import traceback
        print(traceback.format_exc())

if __name__ == "__main__":
    run_gen_ai_course_creation()

True
True

=== Debug: Incoming Prompt ===
Prompt: Create a course for GEN AI from beginner to advance, along with the resources 
    and time durations. Look into MOOCs, Book, YouTube channels, Papers, GitHub

=== Debug: Model Response ===
Response: Creating a comprehensive course for General AI (Artificial Intelligence) from beginner to advanced levels involves a blend of foundational knowledge, technical skills, and understanding of current research and applications. This curriculum will guide you through various stages, utilizing MOOCs (Massive Open Online Courses), books, YouTube channels, research papers, and GitHub repositories as resources. The time durations are approximate and may vary based on individual pace and prior knowledge.

### Beginner Level

**Objective:** Understand the basics of AI, machine learning, and data science.

**Duration:** 3-4 months

**Resources:**

- **MOOCs:**
  - Coursera: "AI For Everyone" by Andrew Ng (4 weeks)
  - edX: "Introduction to Artificial I

In [28]:
from openai import OpenAI
import json

class PromptImprover:
    def __init__(self):
        self.client = OpenAI()
        self.principles = []
        
    def get_principle_from_feedback(self, original_prompt: str, original_response: str, feedback: str) -> str:
        """Convert feedback into a general principle using examples"""
        
        principle_prompt = f"""Given these examples of how feedback improves responses:

Example 1:
INSTRUCTIONS: Write a story
RESULT: "Once upon a time, there was a cucumber..."
REWRITTEN_RESULT: "Once upon a time, there was a cucumber named Carl..."
INSTRUCTION_ADDITION: "Give each character in the story a name"

Example 2:
INSTRUCTIONS: Summarize text
RESULT: "Multiple sentence summary..."
REWRITTEN_RESULT: "One sentence summary..."
INSTRUCTION_ADDITION: "The summary should be a single sentence"

Now for our case:
INSTRUCTIONS: {original_prompt}
RESULT: {original_response}
REWRITTEN_RESULT: {feedback}
Create a generalized addition to add to the INSTRUCTIONS based on the rewritten result:
INSTRUCTION_ADDITION: """

        response = self.client.chat.completions.create(
            model="gpt-4-turbo-preview",
            messages=[{"role": "user", "content": principle_prompt}],
            temperature=0.1
        )
        
        return response.choices[0].message.content

    def improve_prompt(self, original_prompt: str, principle: str) -> str:
        """Apply principle to improve the prompt"""
        
        improvement_prompt = f"""Rewrite the following INSTRUCTIONS so that it adheres to the provided FEEDBACK.
Follow these rules for editing:
1. No change if instruction is already included
2. Preserve current instructions when adding new ones
3. Rewrite existing instruction if new rule changes goals
4. Output should contain entire text of edited prompt

ORIGINAL INSTRUCTIONS: {original_prompt}
FEEDBACK: {principle}
UPDATED INSTRUCTIONS: """

        response = self.client.chat.completions.create(
            model="gpt-4-turbo-preview",
            messages=[{"role": "user", "content": improvement_prompt}],
            temperature=0.1
        )
        
        return response.choices[0].message.content

def example_usage():
    improver = PromptImprover()
    
    # Initial prompt
    initial_prompt = """Create a course for GEN AI from beginner to advance, along with the resources 
    and time durations. Look into MOOCs, Book, YouTube channels, Papers, GitHub"""
    
    print("Initial Prompt:")
    print("="*50)
    print(initial_prompt)
    
    # Get initial response
    response = improver.client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[{"role": "user", "content": initial_prompt}],
        temperature=0.7
    )
    initial_response = response.choices[0].message.content
    
    print("\nInitial Response:")
    print("="*50)
    print(initial_response)
    
    # Your feedback
    feedback = """What are the critical components of determining GEN AI course. 
    Base your prompt on critical components"""
    
    # Generate principle from feedback
    principle = improver.get_principle_from_feedback(
        initial_prompt, 
        initial_response, 
        feedback
    )
    
    print("\nGenerated Principle:")
    print("="*50)
    print(principle)
    
    # Improve prompt using principle
    improved_prompt = improver.improve_prompt(initial_prompt, principle)
    
    print("\nImproved Prompt:")
    print("="*50)
    print(improved_prompt)
    
    # Get improved response
    response = improver.client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[{"role": "user", "content": improved_prompt}],
        temperature=0.7
    )
    improved_response = response.choices[0].message.content
    
    print("\nImproved Response:")
    print("="*50)
    print(improved_response)

if __name__ == "__main__":
    example_usage()

Initial Prompt:
Create a course for GEN AI from beginner to advance, along with the resources 
    and time durations. Look into MOOCs, Book, YouTube channels, Papers, GitHub

Initial Response:
Creating a comprehensive course for learning General AI (Artificial Intelligence) involves covering a wide range of topics, from the basics of machine learning to advanced concepts in AI. Here's a structured pathway from beginner to advanced levels, including various resources such as MOOCs (Massive Open Online Courses), books, YouTube channels, papers, and GitHub repositories. Note that the time duration for each step can vary significantly depending on your background, dedication, and the depth of understanding you wish to achieve.

### Beginner Level

#### Duration: 3-6 Months

1. **Introduction to Python Programming**
   - **Resource:** Automate the Boring Stuff with Python (Book)
   - **Platform:** Udemy (Course by Al Sweigart)

2. **Basic Understanding of AI and Machine Learning**
   - **M